# Pré-processamento

> **Fase 2 - Time 7**

Neste notebook vamos preparar, limpar e transformar os dados brutos em conjuntos de Treino e Teste padronizados para uso de todo o time.

**Metas Técnicas:**
1.  **Engenharia de Features:** Criação da variável `heart_rate_reserve` (Reserva Cardíaca) para normalizar o esforço por idade: 
$$\text{Reserva} = \frac{\text{Thalach}}{220 - \text{Idade}}$$
2.  **Limpeza:** Remove variáveis irrelevantes (`fbs`) identificadas na EDA.
3.  **Codificação:** Transforma categorias em números (One-Hot Encoding).
4.  **Normalização:** Coloca todas as variáveis numéricas na mesma escala (StandardScaler) — *Vital para SVM, KNN e Redes Neurais*.
5.  **Split e Exportação:** Divide os dados (80% Treino / 20% Teste) e salva em arquivos `.pkl` na pasta `data/processed/`.

## 1. Configuração do Ambiente

In [5]:
# Importações
import pandas as pd
import numpy as np
import os
import joblib 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Configuração de Reproducibilidade
SEED = 42

In [6]:
# 1. Carregamento dos Dados
caminho_arquivo = '../data/heart-disease-cleveland-uci/heart_cleveland_upload.csv'

if os.path.exists(caminho_arquivo):
    df = pd.read_csv(caminho_arquivo)
    print(f"✅ Dataset carregado! Shape original: {df.shape}")
else:
    raise FileNotFoundError("❌ Arquivo não encontrado. Verifique o caminho!")

# Visualizar antes
df.head()

✅ Dataset carregado! Shape original: (297, 14)


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,condition
0,69,1,0,160,234,1,2,131,0,0.1,1,1,0,0
1,69,0,0,140,239,0,0,151,0,1.8,0,2,0,0
2,66,0,0,150,226,0,0,114,0,2.6,2,0,0,0
3,65,1,0,138,282,1,2,174,0,1.4,1,1,0,1
4,64,1,0,110,211,0,2,144,1,1.8,1,0,0,0


## 2. Pipeline de Transformação

Utilizaremos um `ColumnTransformer` para aplicar tratamentos diferentes para cada tipo de dado simultaneamente:

* 🔢 **Numéricas:** `StandardScaler` (Age, Trestbps, Chol, Thalach, Oldpeak, Ca).
* 🔠 **Categóricas:** `OneHotEncoder` (Cp, Thal, Slope, Restecg).
* ✅ **Binárias:** `Passthrough` (Sex, Exang) - já estão no formato correto.

In [7]:
# Engenharia de Features

# CRIANDO: 'heart_rate_reserve' (Reserva de Frequência Cardíaca)
# Lógica: Normaliza o esforço baseado na idade. (Thalach / (220 - Idade))
# Isso ajuda o modelo a entender que 150bpm num idoso é muito mais grave que num jovem.
df['heart_rate_reserve'] = df['thalach'] / (220 - df['age'])

# REMOVENDO: 'fbs' (Glicemia)
# Motivo: P-valor de 1.0 na EDA (irrelevante para este dataset de emergência).
df_clean = df.drop(columns=['fbs'])

print("✅ Feature 'heart_rate_reserve' criada.")
print("✅ Coluna 'fbs' removida.")
print(f"Shape atual: {df_clean.shape}")

✅ Feature 'heart_rate_reserve' criada.
✅ Coluna 'fbs' removida.
Shape atual: (297, 14)


In [8]:
# Definição das Transformações

# Separando as variáveis (X) do alvo (y)
X = df_clean.drop('condition', axis=1)
y = df_clean['condition']

# --- DEFINIÇÃO DOS GRUPOS DE VARIÁVEIS ---

# A. Categóricas (Precisam de One-Hot Encoding)
# cp: tipo de dor | restecg: eletro | slope: inclinação | thal: talassemia
categorical_features = ['cp', 'restecg', 'slope', 'thal']

# B. Numéricas (Precisam de StandardScaler)
# Importante para SVM e KNN não darem peso errado para números grandes (como chol)
# Nota: Incluímos 'ca' aqui pois varia de 0 a 3, melhor tratar como numérico escalado para SVM.
numerical_features = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'heart_rate_reserve', 'ca']

# C. Binárias (Já são 0 ou 1, não precisa fazer nada)
binary_features = ['sex', 'exang']

# --- CRIANDO O PROCESSADOR ---

# Pipeline Numérico
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

# Pipeline Categórico
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Juntando tudo no ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features),
        ('pass', 'passthrough', binary_features)
    ],
    verbose_feature_names_out=False 
)

print("Pipeline configurado.")

Pipeline configurado.


## 3. Divisão e Processamento (Split)
Realizaremos a divisão **antes** do processamento para evitar *Data Leakage* (vazamento de dados). O processador aprende a média e desvio padrão apenas do TREINO e aplica essa regra no TESTE.

* **Tamanho do Teste:** 20% (Ideal para dataset pequeno, ~60 amostras de teste).
* **Estratificação:** Garante a mesma proporção de doentes em ambos os conjuntos.

In [9]:
# Train/Test Split

# Primeiro: Dividimos os dados BRUTOS (para garantir que o Teste nunca vazou no Treino)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.20,    # 20% para Teste
    stratify=y,        # Mantém a proporção de doentes igual no treino e teste
    random_state=SEED  # Garante que todo mundo use os mesmos dados
)

# Ajustamos o processador APENAS no Treino (.fit)
preprocessor.fit(X_train_raw)

# Transformamos os dados (.transform)
# O resultado vira um Array Numpy (matriz de números), pronto para os modelos
X_train_processed = preprocessor.transform(X_train_raw)
X_test_processed = preprocessor.transform(X_test_raw)

# Recuperando os nomes das colunas novas (para sabermos quem é quem depois)
# Isso é importante para ver qual feature foi mais importante
feature_names = preprocessor.get_feature_names_out()

# Transformando de volta em DataFrame (opcional, mas ajuda na visualização)
X_train_df = pd.DataFrame(X_train_processed, columns=feature_names)
X_test_df = pd.DataFrame(X_test_processed, columns=feature_names)

print(f"✅ Dados processados com sucesso!")
print(f"Treino: {X_train_df.shape} (80%)")
print(f"Teste:  {X_test_df.shape} (20%)")
print("-" * 30)
print("Features finais:", feature_names)

✅ Dados processados com sucesso!
Treino: (237, 22) (80%)
Teste:  (60, 22) (20%)
------------------------------
Features finais: ['age' 'trestbps' 'chol' 'thalach' 'oldpeak' 'heart_rate_reserve' 'ca'
 'cp_0' 'cp_1' 'cp_2' 'cp_3' 'restecg_0' 'restecg_1' 'restecg_2' 'slope_0'
 'slope_1' 'slope_2' 'thal_0' 'thal_1' 'thal_2' 'sex' 'exang']


## 4. Exportação

In [12]:
# Criando a pasta se não existir
output_path = '../data/processed' 
models_path = '../models/deploy'

os.makedirs(output_path, exist_ok=True)
os.makedirs(models_path, exist_ok=True)

# Dicionário com tudo que precisamos salvar
artifacts = {
    'X_train': X_train_df,
    'X_test': X_test_df,
    'y_train': y_train,
    'y_test': y_test,
    'feature_names': feature_names,
    'preprocessor': preprocessor # Salvamos o transformador caso precisemos usar em dados novos depois
}

# Salvando cada um
for name, data in artifacts.items():
    file_path = os.path.join(output_path, f'{name}.pkl')
    joblib.dump(data, file_path)
    print(f"💾 Salvo: {file_path}")

# B. Salvando Artefatos Críticos para o App (Deploy)
# O app precisa saber como transformar os dados (preprocessor) e a ordem das colunas
joblib.dump(preprocessor, os.path.join(models_path, 'preprocessor.pkl'))
joblib.dump(feature_names, os.path.join(models_path, 'colunas_treino.pkl'))

print("\nTUDO PRONTO!")

💾 Salvo: ../data/processed/X_train.pkl
💾 Salvo: ../data/processed/X_test.pkl
💾 Salvo: ../data/processed/y_train.pkl
💾 Salvo: ../data/processed/y_test.pkl
💾 Salvo: ../data/processed/feature_names.pkl
💾 Salvo: ../data/processed/preprocessor.pkl

TUDO PRONTO!
